<a href="https://colab.research.google.com/github/kumanomi871/sumipe/blob/main/SumipeLineBlogToPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 上坂すみれ公式LINEブログをPDF化し、保存するプログラム
上阪すみれオフィシャルブログ(https://lineblog.me/uesaka_sumire/) の1ページから300ページまでPDF化し、Google Driveに保存します


#### 実行手順
1.   最初のセルのシェルスクリプトを実行してください 
(スクリプトの内容は以下をご覧ください https://teratail.com/questions/9wfzcz4dxd22m1)
2.   2番目のセルを実行すると保存が開始されます　







In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver fonts-noto

In [ ]:
!pip install selenium
!pip install webdriver_manager


import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import os
import base64


def save_to_pdf(driver, file_path):
    parameters = {
        "printBackground": True, # 背景画像を印刷
        "paperWidth": 8.27, # A4用紙の横 210mmをインチで指定
        "paperHeight": 11.69, # A4用紙の縦 297mmをインチで指定
        # "displayHeaderFooter": True, # 印刷時のヘッダー、フッターを表示
    }
    # Chrome Devtools Protocolコマンドを実行し、取得できるBase64形式のPDFデータをデコードしてファイルに保存
    pdf_base64 = driver.execute_cdp_cmd("Page.printToPDF", parameters)
    # print(pdf_base64)
    pdf = base64.b64decode(pdf_base64["data"])
    with open(file_path, 'bw') as f:
        f.write(pdf)


# google driveを連携
from google.colab import drive
drive.mount('./gdrive')

# 画像の保存先ディレクトリの作成と指定
import os
google_drive_save_dir = "./gdrive/My Drive/Uesaka Sumire/lineblogPDFs"
os.makedirs(google_drive_save_dir, exist_ok=True)

# カレントディレクトリにすみぺLINEBLOGのURLリスト作成
root_url = 'https://lineblog.me/uesaka_sumire/'
urls = [root_url]

for i, n_page in enumerate(range(2,301,1),1):
    page_url = root_url + "?p={}".format(n_page)
    urls.append(page_url)
    
df = pd.DataFrame(urls)
df.to_csv('urls.txt', index=False, header=False)

# URLリスト取得
urls = []
with open('urls.txt', mode='rt', encoding='utf-8') as f:
    urls = f.readlines()

# Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')

# PDF保存実行
driver = webdriver.Chrome(service=ChromeService("/usr/bin/chromedriver"),options=options)
print('\nPDFダウンロード開始') 
for i, url in enumerate(urls,1):
  driver.get(url)
  file_name = "{}.pdf".format(i)
  pdf_path = os.path.join(google_drive_save_dir, file_name)
  print(pdf_path)
  save_to_pdf(driver, pdf_path)

driver.close()
driver.quit()